In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/marco/anaconda3/bin/python
3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


In [2]:
sc.stop()

In [3]:
#sc.stop()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("Feature Enginering")\
    .getOrCreate()

sc = spark.sparkContext

In [4]:
sc._conf.get('spark.driver.memory')

'16g'

In [5]:
sc._conf.getAll()

[('spark.master', 'local[4]'),
 ('spark.local.dir', '/home/marco/claseBigData/ProyectoBD/tmp'),
 ('spark.app.startTime', '1638933793094'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.host', '192.168.3.5'),
 ('spark.app.name', 'Feature Enginering'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1638933793171'),
 ('spark.driver.port', '43561'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
sc.version

'3.0.3'

In [7]:
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/marco/anaconda3/bin/python
3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


![imagenes/](img/DataPreparation.png)

#### Carga de fuentes de información

Cabeceras:
    - origen: numero de origen,
    - destino: numero de destino,
    - tipo: tipo de llamada,
    - segundos: duración de llamada (seg),
    - llamadas: numero de llmadas, 
    - mensajes: numero de sms

In [8]:
from pyspark.sql.types import StructField, StructType, StringType,DoubleType
custom_schema = StructType([
    StructField("origen", StringType(), True),
    StructField("destino", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("segundos", DoubleType(), True),
    StructField("llamadas", DoubleType(), True),
    StructField("mensajes", DoubleType(), True)
])

mtDf = spark.read.format("csv")\
        .schema(custom_schema)\
        .option("header","false")\
        .load("../rawData/mt/week2/20150310.csv.sanitaized")

mtDf.show(truncate=True)

mtDf.filter(mtDf.tipo!="MOBILE").show()

In [9]:
moDf = spark.read.format("csv")\
        .schema(custom_schema)\
        .option("header","false")\
        .load("../rawData/mo/week2/20150310.csv")

In [10]:
moDf.show()

+--------------------+--------------------+-------+--------+--------+--------+
|              origen|             destino|   tipo|segundos|llamadas|mensajes|
+--------------------+--------------------+-------+--------+--------+--------+
|dad878015fcbecfda...|261d09ee269bd79c0...| MOBILE|    23.0|     2.0|     0.0|
|3b3af9f22ec61f89c...|f228da1389ba37cc1...| MOBILE|    57.0|     1.0|     0.0|
|840addc6fb560bd0a...|54417a8d60abcafb8...| MOBILE|    52.0|     1.0|     0.0|
|696e2cd1f2874f0ec...|02782fd0428887eda...| MOBILE|    35.0|     2.0|     0.0|
|e5f9dfeb9b97f392e...|7ed5dc16f1b74c11d...| MOBILE|   379.0|     1.0|     0.0|
|ecaf1439ea61a47c6...|65f09eaaafd0b7275...| MOBILE|   228.0|     2.0|     0.0|
|cbb5f445dff5ec7ca...|84a64468e4b26872b...| MOBILE|    42.0|     2.0|     0.0|
|5b61a1d5d08bfbccf...|a1ced1b13747c0cc9...| MOBILE|    43.0|     2.0|     0.0|
|178f76bc7bc5ad28e...|484342a23fc025e82...| MOBILE|   141.0|     2.0|     0.0|
|243a442812e365849...|12b104daa5924ec49...| MOBILE| 

In [11]:
#moDf.filter(moDf.tipo!="MOBILE").count()   # 994,258

In [12]:
#moDf.filter(moDf.tipo=="MOBILE").count()   # 7,151,552

In [13]:
from pyspark.sql import functions as F

In [14]:
#moDf.groupBy("origen").agg(F.sum("segundos").alias("sum_segundos"), \
 #                          F.countDistinct("destino").alias("destinos"), \
  #                         F.sum("llamadas").alias("sum_llamadas"), \
   #                        F.sum("mensajes").alias("sum_sms")).show()

In [15]:
moDf.groupBy("tipo").agg(F.count("*"),F.countDistinct(moDf.origen)).show()

+-------------+--------+-------------+
|         tipo|count(1)|count(origen)|
+-------------+--------+-------------+
|     LANDLINE|  189466|       162658|
|       OTHERS|    2720|         2466|
|      SERVICE|  714568|       632290|
|       MOBILE| 7151552|      1986899|
|INTERNATIONAL|   87504|        64002|
+-------------+--------+-------------+



In [16]:
#moDf.groupBy("origen","tipo").agg(F.when(F.col("tipo")=="MOBILE",F.sum("segundos")).otherwise(0).alias("sum_segundos")\
#                          ).show()

In [17]:
moDfInter01=moDf\
    .withColumn("segMobile", F.when(moDf.tipo == "MOBILE", moDf.segundos).otherwise(0))\
    .withColumn("segService", F.when(moDf.tipo == "SERVICE", moDf.segundos).otherwise(0))\
    .withColumn("segInternational", F.when(moDf.tipo == "INTERNATIONAL", moDf.segundos).otherwise(0))\
    .withColumn("segOthers", F.when(~((moDf.tipo=="INTERNATIONAL")|\
                                            (moDf.tipo=="SERVICE")|\
                                            (moDf.tipo=="MOBILE")), moDf.segundos).otherwise(0))\
    .withColumn("llaMobile", F.when(moDf.tipo == "MOBILE", moDf.llamadas).otherwise(0))\
    .withColumn("llaService", F.when(moDf.tipo == "SERVICE", moDf.llamadas).otherwise(0))\
    .withColumn("llaInternational", F.when(moDf.tipo == "INTERNATIONAL", moDf.llamadas).otherwise(0))\
    .withColumn("llaOthers", F.when(~((moDf.tipo=="INTERNATIONAL")|\
                                            (moDf.tipo=="SERVICE")|\
                                            (moDf.tipo=="MOBILE")), moDf.llamadas).otherwise(0))\
    .withColumn("desMobile", F.when(moDf.tipo == "MOBILE", moDf.destino))\
    .withColumn("desService", F.when(moDf.tipo == "SERVICE", moDf.destino))\
    .withColumn("desInternational", F.when(moDf.tipo == "INTERNATIONAL", moDf.destino))\
    .withColumn("desOthers", F.when(~((moDf.tipo=="INTERNATIONAL")|\
                                            (moDf.tipo=="SERVICE")|\
                                            (moDf.tipo=="MOBILE")), moDf.destino))

In [18]:
moDfInter01.count()

8145810

In [19]:
moDfInter01 = moDfInter01.groupBy("origen")\
            .agg(\
                 ### Agregando en Segundos
                 F.sum("segundos").alias("sumSeg")\
                ,F.sum("segMobile").alias("sumSegMobile")\
                ,F.sum("segService").alias("sumSegService")\
                ,F.sum("segInternational").alias("sumSegInternational")\
                ,F.sum("segOthers").alias("sumSegOthers")\
                 ### Agregando en llamadas
                ,F.sum("llamadas").alias("sumLla")\
                ,F.sum("llaMobile").alias("sumLlaMobile")\
                ,F.sum("llaService").alias("sumLlaService")\
                ,F.sum("llaInternational").alias("sumLlaInternational")\
                ,F.sum("llaOthers").alias("sumLlaOthers")\
                 ### Agregando destinos distintos
                ,F.countDistinct("destino").alias("nroDest")\
                ,F.countDistinct("desMobile").alias("nroDestMobile")\
                ,F.countDistinct("desService").alias("nroDestService")\
                ,F.countDistinct("desInternational").alias("nroDestInternational")\
                ,F.countDistinct("desOthers").alias("nroDestOthers")\
                )

In [20]:
moDfInter01.count()

2210592

In [21]:
moDfInter01.limit(10).toPandas().head()

,origen,sumSeg,sumSegMobile,sumSegService,sumSegInternational,sumSegOthers,sumLla,sumLlaMobile,sumLlaService,sumLlaInternational,sumLlaOthers,nroDest,nroDestMobile,nroDestService,nroDestInternational,nroDestOthers
0,3aba283ea13cf7d2380cce878687bd092aafed8f89edee...,150.0,150.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,0,0,0
1,b21833971e53bee50dab425fbab163628ec1831a004b76...,1257.0,1257.0,0.0,0.0,0.0,27.0,27.0,0.0,0.0,0.0,9,9,0,0,0
2,29e3d2f9452df3765cdc54c2b06b91ee1dc5e9057b0333...,1810.0,1707.0,103.0,0.0,0.0,14.0,12.0,2.0,0.0,0.0,10,9,1,0,0
3,4987165ca4239c6d955e9d1e1eca74ec3215ff4b1ab9d2...,33.0,33.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,0,0,0
4,f9ba9adce15a3b2246cb6ef23ac9902591ae03dd8bf84d...,8122.0,8122.0,0.0,0.0,0.0,16.0,16.0,0.0,0.0,0.0,6,6,0,0,0


In [22]:
moDf.filter(moDf.origen=="5f96f009c808466497ce0a5e19126cd9144be71c3245766948a57a2e").show()

+--------------------+--------------------+-------------+--------+--------+--------+
|              origen|             destino|         tipo|segundos|llamadas|mensajes|
+--------------------+--------------------+-------------+--------+--------+--------+
|5f96f009c80846649...|c4b73c9223bac76b8...|       OTHERS|   133.0|     1.0|     0.0|
|5f96f009c80846649...|f110df29d30cb3b7f...|INTERNATIONAL|   429.0|     1.0|     0.0|
|5f96f009c80846649...|919d4f8ff96c308ea...|       MOBILE|    22.0|     1.0|     0.0|
|5f96f009c80846649...|9795116863458d126...|       MOBILE|    19.0|     1.0|     0.0|
|5f96f009c80846649...|abe25dcd1d6e8b186...|     LANDLINE|   262.0|     1.0|     0.0|
+--------------------+--------------------+-------------+--------+--------+--------+



In [23]:
### Agregando Proporciones
### Agregando proporciones en segundos con un artificio para evitar la división por 0
### Agregando proporciones en llamadas con un artificio para evitar la división por 0
### Agregando proporciones en nro de destinos con un artificio para evitar la división por 0
moDfInter01 = moDfInter01\
    .withColumn("PropSegMobile", F.round((moDfInter01.sumSegMobile+0.5)/(moDfInter01.sumSeg+1),5))\
    .withColumn("PropSegService", F.round((moDfInter01.sumSegService+0.5)/(moDfInter01.sumSeg+1),5))\
    .withColumn("PropSegInternational", F.round((moDfInter01.sumSegInternational+0.5)/(moDfInter01.sumSeg+1),5))\
    .withColumn("PropLlaMobile", F.round((moDfInter01.sumLlaMobile+0.5)/(moDfInter01.sumLla+1),5))\
    .withColumn("PropLlaService", F.round((moDfInter01.sumLlaService+0.5)/(moDfInter01.sumLla+1),5))\
    .withColumn("PropLlaInternational", F.round((moDfInter01.sumLlaInternational+0.5)/(moDfInter01.sumLla+1),5))\
    .withColumn("PropNroDestMobile", F.round((moDfInter01.nroDestMobile+0.5)/(moDfInter01.nroDest+1),5))\
    .withColumn("PropNroDestService", F.round((moDfInter01.nroDestService+0.5)/(moDfInter01.nroDest+1),5))\
    .withColumn("PropNroDestInternational", F.round((moDfInter01.nroDestInternational+0.5)/(moDfInter01.nroDest+1),5))

In [24]:
moDfInter01.limit(10).toPandas().head()

,origen,sumSeg,sumSegMobile,sumSegService,sumSegInternational,sumSegOthers,sumLla,sumLlaMobile,sumLlaService,sumLlaInternational,...,nroDestOthers,PropSegMobile,PropSegService,PropSegInternational,PropLlaMobile,PropLlaService,PropLlaInternational,PropNroDestMobile,PropNroDestService,PropNroDestInternational
0,3aba283ea13cf7d2380cce878687bd092aafed8f89edee...,150.0,150.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0.99669,0.00331,0.00331,0.75000,0.25000,0.25000,0.75000,0.25000,0.25000
1,b21833971e53bee50dab425fbab163628ec1831a004b76...,1257.0,1257.0,0.0,0.0,0.0,27.0,27.0,0.0,0.0,...,0,0.99960,0.00040,0.00040,0.98214,0.01786,0.01786,0.95000,0.05000,0.05000
2,29e3d2f9452df3765cdc54c2b06b91ee1dc5e9057b0333...,1810.0,1707.0,103.0,0.0,0.0,14.0,12.0,2.0,0.0,...,0,0.94285,0.05715,0.00028,0.83333,0.16667,0.03333,0.86364,0.13636,0.04545
3,4987165ca4239c6d955e9d1e1eca74ec3215ff4b1ab9d2...,33.0,33.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0.98529,0.01471,0.01471,0.75000,0.25000,0.25000,0.75000,0.25000,0.25000
4,f9ba9adce15a3b2246cb6ef23ac9902591ae03dd8bf84d...,8122.0,8122.0,0.0,0.0,0.0,16.0,16.0,0.0,0.0,...,0,0.99994,0.00006,0.00006,0.97059,0.02941,0.02941,0.92857,0.07143,0.07143


In [25]:
moDfInter01.count()

2210592

In [26]:
moDfInter01.coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/mo_variables_iniciales.parquet")


### Contactos distintos

In [27]:
moDf.select("origen","destino").coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/mo_destinos.parquet")